#Model Building

In [3]:
import pandas as pd
data=pd.read_csv('/content/drive/MyDrive/comments/new_master_data.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4,14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


**This new data contains the augmented data as well which i have generated by using transformers models**

In [4]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
import re

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
normal_comments=data[(((((data['malignant']!=1) &(data['highly_malignant']!=1) & (data['rude']!=1) & (data['threat']!=1) & (data['abuse']!=1) & (data['loathe']!=1)))))]

as i have a large no of dataset and my system is not able to handle that much thats why i am selecting the normal comments there

In [6]:
normal_comments_data=normal_comments.sample(n=100000)

here i have selected 100000 rows as a normal data that have only 0 value

In [7]:
malignant=data[data['malignant']==1]
highly_malignant=data[data['highly_malignant']==1]
rude=data[data['rude']==1]
threat=data[data['threat']==1]
abuse=data[data['abuse']==1]
loathe=data[data['loathe']==1]

In [8]:
frames=[normal_comments_data,malignant,rude,threat,abuse,loathe]

In [9]:
new_data=pd.concat(frames)
new_data.drop(['Unnamed: 0','level_0','index'],axis=1,inplace=True)

Here i have concat the all labels values these value have the augmented data as well

In [10]:
data = new_data.sample(frac=1).reset_index(drop=True)

In [11]:
data

,id,comment_text,malignant,highly_malignant,rude,threat,abuse,loathe,Unnamed: 0.1,clean_comment_text,Unnamed: 0.1.1,Labels,encoded_cat
0,0,"I also make mistakes, but I'm not a fucking va...",1.0,0.0,1.0,0.0,1.0,0.0,94247.0,also make mistake fucking vandal told hate cal...,94247.0,"malignant,rude,abuse",23.0
1,0,"That's next day, so I might as well see Main P...",0.0,0.0,0.0,0.0,0.0,0.0,81695.0,next day might well see main page,81695.0,Normal,0.0
2,0,That's unquestionably unfair. I've seen loads ...,0.0,0.0,0.0,0.0,0.0,0.0,105454.0,unquestionably unfair seen load screenshots pe...,105454.0,Normal,0.0
3,0,u motherfukkin bitch i want to rape you smelly...,1.0,1.0,1.0,1.0,1.0,1.0,29968.0,u motherfukkin bitch want rape smelly whore st...,29968.0,"malignant,highly_malignant,rude,threat,abuse,l...",17.0
4,0,It's not even on television yet.,0.0,0.0,0.0,0.0,0.0,0.0,115596.0,even television yet,115596.0,Normal,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
178212,0,0,0.0,0.0,0.0,0.0,0.0,1.0,12194.0,daedalus nothing filthy n gger cyberstalker si...,0.0,0,0.0
178213,0,0,0.0,0.0,0.0,0.0,0.0,1.0,11519.0,I'm wondering if faggot large participate sexu...,0.0,0,0.0
178214,0,"""\n\n further move needed on reviewed article ...",0.0,0.0,0.0,0.0,0.0,0.0,36206.0,move needed reviewed article hi thanks reviewi...,36206.0,Normal,0.0
178215,0,0,0.0,0.0,0.0,0.0,0.0,1.0,12452.0,Is it possible that people god behold nice thi...,0.0,0,0.0


In [ ]:
#wnl=WordNetLemmatizer()
#corpus=[]

"""for i in range(len(data)):
  review=re.sub('[^a-zA-Z]',' ',data['clean_comment_text'][i])
  review=review.lower()
  review=review.split()
  review=[wnl.lemmatize(word) for word in review if not word in stopwords.words('english')]
  review=' '.join(review)
  corpus.append(review)"""

In [ ]:
#data['clean_comment_text']=corpus

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vec=TfidfVectorizer(ngram_range=(1,2),
                          min_df=2,
                          max_features=15000)
tfidf_vec.fit(data['clean_comment_text'])

# trasform train and test
train_tfidf = tfidf_vec.transform(data['clean_comment_text'])

Here i am using tfidf for converting my text data into numeric

In [13]:
from sklearn.model_selection import train_test_split,cross_val_score,StratifiedKFold,GridSearchCV
from sklearn.metrics import accuracy_score,classification_report,f1_score,roc_auc_score

In [14]:
list_classes = ['malignant','highly_malignant','rude','threat','abuse','loathe']
train_y = data[list_classes].values
X_train,X_test,y_train,y_test=train_test_split(train_tfidf,train_y,test_size=0.3,random_state=42)

In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier

In [48]:
models={"RandomForestClassifier":RandomForestClassifier(),
        "DecisionTreeClassifier":DecisionTreeClassifier(),
        "ExtraTreesClassifier":ExtraTreesClassifier(),
        "KNeighborsClassifier":KNeighborsClassifier()}

In [49]:
for name,model in models.items():
  print("****************************************************",name,"*********************************************************")
  print('\n')
  model.fit(X_train,y_train)
  print(model)
  y_pred=model.predict(X_test)
  print('\n')
  acc=accuracy_score(y_test,y_pred)
  print("Accuracy Score",acc)
  print('\n')
  F1_SCORE=f1_score(y_test,y_pred,average='micro')
  print("F1 SCORE",F1_SCORE)
  report=classification_report(y_test,y_pred)
  print('\n')
  print("CLASSIFICATION REPORT")
  print("\n")
  print(report)
  roc=roc_auc_score(y_test,y_pred)
  print("ROC AUC SCORE",roc)

**************************************************** RandomForestClassifier *********************************************************


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)


Accuracy Score 0.8168555717652339


F1 SCORE 0.8069438358635976


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))




CLASSIFICATION REPORT


              precision    recall  f1-score   support

           0       0.90      0.80      0.85      9702
           1       0.95      0.87      0.91      1486
           2       0.80      0.74      0.77      9628
           3       0.89      0.88      0.89      4831
           4       0.78      0.72      0.75      9123
           5       0.85      0.76      0.80      5672

   micro avg       0.84      0.77      0.81     40442
   macro avg       0.86      0.79      0.83     40442
weighted avg       0.84      0.77      0.81     40442
 samples avg       0.28      0.29      0.28     40442

ROC AUC SCORE 0.8865871666285526
**************************************************** DecisionTreeClassifier *********************************************************


DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


ROC AUC SCORE 0.8759874021557551
**************************************************** ExtraTreesClassifier *********************************************************


ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=100,
                     n_jobs=None, oob_score=False, random_state=None, verbose=0,
                     warm_start=False)


Accuracy Score 0.8183518497736879


F1 SCORE 0.8089456368478772


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))




CLASSIFICATION REPORT


              precision    recall  f1-score   support

           0       0.91      0.78      0.84      9702
           1       0.96      0.86      0.91      1486
           2       0.81      0.73      0.77      9628
           3       0.90      0.88      0.89      4831
           4       0.79      0.71      0.75      9123
           5       0.88      0.77      0.82      5672

   micro avg       0.86      0.77      0.81     40442
   macro avg       0.88      0.79      0.83     40442
weighted avg       0.86      0.77      0.81     40442
 samples avg       0.28      0.28      0.28     40442

ROC AUC SCORE 0.8848453876020378
**************************************************** KNeighborsClassifier *********************************************************


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')


Accuracy Score 0.6909

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**I will select extrastree as my best model**

**Note=**I was training many model but that models was giving me error because this Y value have multilabel so just because of that i was not able to train my dataset on that models

In [16]:
Extra=ExtraTreesClassifier()

In [17]:
Extra.fit(X_train,y_train)

ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=100,
                     n_jobs=None, oob_score=False, random_state=None, verbose=0,
                     warm_start=False)

In [18]:
pre=Extra.predict(X_test)

In [19]:
accuracy_score(y_test,pre)

0.8186698088504845

#Laoding Test Data

In [20]:
test=pd.read_csv('/content/drive/MyDrive/datatraine/test.csv')

#Cleaning Test Data

In [21]:
wnl=WordNetLemmatizer()
corpus=[]

for i in range(len(test)):
  review=re.sub('[^a-zA-Z]',' ',test['comment_text'][i])
  review=review.lower()
  review=review.split()
  review=[wnl.lemmatize(word) for word in review if not word in stopwords.words('english')]
  review=' '.join(review)
  corpus.append(review)

In [22]:
test['clean_comment_text']=corpus

#Feature Transformation

In [23]:
tfidf_vec = TfidfVectorizer(ngram_range=(1,2), 
                            min_df=2, 
                            max_features=15000)
tfidf_vec.fit(test['clean_comment_text'])

# trasform train and test
test_tfidf = tfidf_vec.transform(test['clean_comment_text'])

In [26]:
predictions=Extra.predict(test_tfidf)

In [27]:
pre_data=pd.DataFrame(predictions,columns=list_classes)

In [29]:
test_data_comments=pd.concat([pre_data,test['clean_comment_text']],axis=1)

#The result of prediction of malignant comment

In [34]:
test_data_comments[test_data_comments['malignant']==1]

,malignant,highly_malignant,rude,threat,abuse,loathe,clean_comment_text
14,1.0,0.0,0.0,0.0,0.0,0.0,adding new product list make sure relevant add...
41,1.0,0.0,0.0,0.0,0.0,0.0,convinced blind documented possible call legal...
75,1.0,0.0,0.0,0.0,1.0,0.0,ridiculous unless good non disingenuous respon...
100,1.0,0.0,1.0,0.0,0.0,0.0,could precis material instead downloading whol...
104,1.0,0.0,0.0,0.0,0.0,0.0,catdiffuse think best idea would check whatlin...
...,...,...,...,...,...,...,...
153038,1.0,0.0,0.0,0.0,0.0,0.0,rate article definitely need work capitalizati...
153061,1.0,0.0,0.0,0.0,0.0,0.0,sarah anyone edit page anyone nominate deletio...
153106,1.0,0.0,1.0,0.0,0.0,0.0,youshit dick cock fuckshit dick cock fuck shit...
153116,1.0,0.0,1.0,0.0,0.0,0.0,utc fyi currently edit conflict user bakasuprm...


#Highly_malignant comment

In [35]:
test_data_comments[test_data_comments['highly_malignant']==1]

,malignant,highly_malignant,rude,threat,abuse,loathe,clean_comment_text
62046,1.0,1.0,1.0,0.0,1.0,0.0,p user also editing people talk page
67203,1.0,1.0,1.0,0.0,1.0,0.0,changing people talk page comment


#Abuse Comment

In [36]:
test_data_comments[test_data_comments['abuse']==1]

,malignant,highly_malignant,rude,threat,abuse,loathe,clean_comment_text
75,1.0,0.0,0.0,0.0,1.0,0.0,ridiculous unless good non disingenuous respon...
106,0.0,0.0,0.0,0.0,1.0,0.0,avg plenty greek love king stop acting like gr...
288,1.0,0.0,1.0,0.0,1.0,0.0,recommendation point keep stuff main space
308,1.0,0.0,1.0,0.0,1.0,0.0,february utc certainly would ask include somet...
382,1.0,0.0,1.0,0.0,1.0,0.0,related page fyi exists draft draft trumpler c...
...,...,...,...,...,...,...,...
152654,1.0,0.0,0.0,0.0,1.0,0.0,page movement hello jk getting impression thar...
152792,1.0,0.0,1.0,0.0,1.0,0.0,tripped keyboard oops tripped keyboard wrote t...
153041,0.0,0.0,1.0,0.0,1.0,0.0,closed captioning read nigger
153090,0.0,0.0,1.0,0.0,1.0,0.0,dear sir fucking cunt rahm emanuel satannnnnnn...


#Conclusion

1-Did Cleaning of the Data

2-Did EDA on the dataset

3-Did Feature Engineering on data 

4-Did data augmentation because most of the labels was only having 500 or 1000 so my model was not able to learn properly

5-Trained Multiple Models

6-Select ExtraTreeClassifier as my Best Model